# ติดตั้งก่อนใช้ EasyOCR

- . /home/conda/etc/profile.d/conda.sh
- conda create -n ocr python=3.11 -y
- pip install easyocr opencv-python pandas deep-translator ipykernel
- pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu126

# สแกนเอกสาร ไทย-อังกฤษ

In [ ]:
import easyocr, cv2

reader = easyocr.Reader(['th', 'en'], gpu=True)  # ภาษาไทย + อังกฤษ
image_file = 'receipt.jpg'
results = reader.readtext(image_file)

# แสดงผลลัพธ์
for _, text, conf in results:
    print(f"{text} (Confidence: {conf:.2f})")
# วาดกรอบรอบข้อความที่ตรวจจับได้
image = cv2.imread(image_file)
for (bbox, text, conf) in results:
    (top_left, top_right, bottom_right, bottom_left) = bbox
    top_left = tuple(map(int, top_left))
    bottom_right = tuple(map(int, bottom_right))
    cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
cv2.imwrite("ocr_output.jpg", image)

# ตัวอย่างการใช้งาน OCR

In [ ]:
# แปลเอกสารไทย-อังกฤษ
from deep_translator import GoogleTranslator
translator = GoogleTranslator(source='th', target='en')
for _, text, conf in results:
    translated = translator.translate(text)
    print(f"ก่อนแปล: {text}，หลังแปล: {translated}")

In [ ]:
import pandas as pd
import re

# จัดเรียงข้อความจากบนลงล่าง ตามตำแหน่งแนวตั้ง
results_sorted = sorted(results, key=lambda x: x[0][0][1])
# เก็บเฉพาะข้อความที่ความมั่นใจมากกว่า 0.4
lines = [text.strip() for _, text, conf in results_sorted if conf > 0.4]
# สร้างรายการสินค้า โดยจัดกลุ่มทุก 5 บรรทัด
items = []
for i in range(0, len(lines), 5):
    group = lines[i:i+5]
    print(f"กลุ่มที่ {i//5 + 1}: {group}")
    if len(group) < 5:
        continue  # ถ้ากลุ่มไม่ครบ 5 บรรทัด ให้ข้ามไป
    try:
        item = {
            'ชื่อสินค้า': group[0],
            'จำนวน': int(group[1]),
            'ราคาต่อหน่วย': float(group[2].replace(',', '')),
            'ส่วนลด': group[3],
            'รวม (บาท)': float(group[4].replace(',', ''))
        }
        items.append(item)
    except ValueError:
        continue  # ข้ามกลุ่มที่มีข้อมูลผิดพลาด เช่น แปลงเป็นตัวเลขไม่ได้

# แสดงผลลัพธ์รายการสินค้าใน Console
print("\n✅ รายการสินค้า:")
for item in items:
    print(f"{item['ชื่อสินค้า']:<30} x{item['จำนวน']:>3} @ {item['ราคาต่อหน่วย']:>8.2f} → {item['รวม (บาท)']:>10,.2f}")

# บันทึกเป็นไฟล์ CSV
df = pd.DataFrame(items)
df.to_csv('receipt_output.csv', index=False)
print("\n📁 บันทึกเป็น receipt_grouped_output.csv เรียบร้อยแล้ว")